# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [79]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [80]:
csv_file = pd.read_csv("../WeatherPy/output_data/cities.csv") 
# csv_file

cities_dataframe = pd.DataFrame(csv_file)

cities_dataframe.head() 


,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
0,Nanortalik,60.14,-45.24,45.41,89,100,6.78,GL,1596493691
1,Wahiawā,21.50,-158.02,87.80,57,75,10.29,US,1596493692
2,Shelburne,44.08,-80.20,68.00,94,40,4.70,CA,1596493692
3,Coquimbo,-29.95,-71.34,53.60,93,40,11.41,CL,1596493692
4,Teeli,51.03,90.23,50.52,73,3,1.21,RU,1596493692


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [81]:
# configure gmaps
gmaps.configure(api_key=g_key) 

# use Lat and Lng as locations
locations = cities_dataframe[["Lat", "Lng"]] 

humidity = cities_dataframe["Humidity (%)"].astype(float)



In [82]:
# Humidity as weight
# Plot Heatmap
fig = gmaps.figure()

# # Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)


# # Add layer
fig.add_layer(heat_layer)

# # Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [83]:
# use loc to find ideal weather for vacation spots
vacation_df = cities_dataframe.loc[cities_dataframe['Max Temp (F)'] > 75, :]

# vacation_df = cities_dataframe.loc[cities_dataframe['Max Temp (F)'] < 95, :]
# vacation_df = cities_dataframe.loc[cities_dataframe['Humidity (%)'] < 90, :]
# vacation_df = cities_dataframe.loc[cities_dataframe['Cloudiness (%)'] < 10, :]
# vacation_df = cities_dataframe.loc[cities_dataframe['Wind Speed (mph)'] < 10, :]

vacation_df = vacation_df.loc[vacation_df['Max Temp (F)'] < 85, :] 
vacation_df = vacation_df.loc[vacation_df['Humidity (%)'] < 50, :] 
vacation_df = vacation_df.loc[vacation_df['Cloudiness (%)'] < 10, :]
vacation_df = vacation_df.loc[vacation_df['Wind Speed (mph)'] < 10, :] 

vacation_df 


,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date
54,Corigliano Calabro,39.59,16.52,78.01,40,0,5.99,IT,1596493705
184,Brewster,41.40,-73.62,84.99,48,1,5.82,US,1596493736
192,Ubari,26.59,12.78,84.22,14,0,7.07,LY,1596493739
232,Vilhena,-12.74,-60.15,77.68,31,0,2.73,BR,1596493751
235,Bethel,41.37,-73.41,84.99,48,1,5.82,US,1596493752
303,Rome,43.21,-75.46,82.40,48,1,3.74,US,1596493772
324,Nortelândia,-14.45,-56.80,80.89,25,0,3.22,BR,1596493777
344,Barra do Garças,-15.89,-52.26,77.14,31,0,4.52,BR,1596493783
436,Gazojak,41.19,61.40,80.15,23,0,4.90,TM,1596493809
490,Bereket,39.24,55.52,82.71,46,1,4.50,TM,1596493822


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [86]:
# rename dataframe to hotel_df
hotel_df = vacation_df

# add column with no data currently 
# https://www.kite.com/python/answers/how-to-create-an-empty-column-in-a-pandas-dataframe-in-python#:~:text=Use%20an%20empty%20string%20to,named%20new_column%20in%20the%20Dataframe%20.
hotel_df["Hotel Name"] = ""  
# hotel_df



# # set params
# # geocoordinates
# # https://developers.google.com/places/web-service/supported_types?hl=en_US
# target_coordinates = 
target_search = "Hotel"
target_radius = 5000
target_type = "lodging"

for index, row in hotel_df.iterrows():
    params = {
    "location": f"{row['Lat']}, {row['Lng']}",
    "keyword": target_search,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
    } 
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    response = requests.get(base_url, params=params).json()
    
#     extract results
    results = response['results']
    
    try:
        print(f"The Hotel to stay at is {results[0]['name']}")
        
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Missing field/result...skipping") 
              

        
    
  
    


The Hotel to stay at is Chimento Resort
The Hotel to stay at is Comfort Inn
The Hotel to stay at is فندق أوباري
The Hotel to stay at is Hotel Mirage
The Hotel to stay at is Best Western Danbury/Bethel
The Hotel to stay at is Hampton Inn Rome
The Hotel to stay at is Hotel Karysmã
The Hotel to stay at is Hilton Hotel
Missing field/result...skipping
Missing field/result...skipping
The Hotel to stay at is Howard Johnson Hotel And Casino Formosa


In [87]:
hotel_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
54,Corigliano Calabro,39.59,16.52,78.01,40,0,5.99,IT,1596493705,Chimento Resort
184,Brewster,41.40,-73.62,84.99,48,1,5.82,US,1596493736,Comfort Inn
192,Ubari,26.59,12.78,84.22,14,0,7.07,LY,1596493739,فندق أوباري
232,Vilhena,-12.74,-60.15,77.68,31,0,2.73,BR,1596493751,Hotel Mirage
235,Bethel,41.37,-73.41,84.99,48,1,5.82,US,1596493752,Best Western Danbury/Bethel
303,Rome,43.21,-75.46,82.40,48,1,3.74,US,1596493772,Hampton Inn Rome
324,Nortelândia,-14.45,-56.80,80.89,25,0,3.22,BR,1596493777,Hotel Karysmã
344,Barra do Garças,-15.89,-52.26,77.14,31,0,4.52,BR,1596493783,Hilton Hotel
436,Gazojak,41.19,61.40,80.15,23,0,4.90,TM,1596493809,
490,Bereket,39.24,55.52,82.71,46,1,4.50,TM,1596493822,


In [91]:
# drop the null values in the hotel_df
hotel_updated_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
hotel_updated_df

,City,Lat,Lng,Max Temp (F),Humidity (%),Cloudiness (%),Wind Speed (mph),Country,Date,Hotel Name
54,Corigliano Calabro,39.59,16.52,78.01,40,0,5.99,IT,1596493705,Chimento Resort
184,Brewster,41.40,-73.62,84.99,48,1,5.82,US,1596493736,Comfort Inn
192,Ubari,26.59,12.78,84.22,14,0,7.07,LY,1596493739,فندق أوباري
232,Vilhena,-12.74,-60.15,77.68,31,0,2.73,BR,1596493751,Hotel Mirage
235,Bethel,41.37,-73.41,84.99,48,1,5.82,US,1596493752,Best Western Danbury/Bethel
303,Rome,43.21,-75.46,82.40,48,1,3.74,US,1596493772,Hampton Inn Rome
324,Nortelândia,-14.45,-56.80,80.89,25,0,3.22,BR,1596493777,Hotel Karysmã
344,Barra do Garças,-15.89,-52.26,77.14,31,0,4.52,BR,1596493783,Hilton Hotel
540,Formosa,-26.18,-58.18,75.20,47,0,8.05,AR,1596493691,Howard Johnson Hotel And Casino Formosa


In [92]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_updated_df.iterrows()]
locations = hotel_updated_df[["Lat", "Lng"]]

In [93]:
# Add marker layer ontop of heat map 

hotel_markers = gmaps.marker_layer(locations)

fig.add_layer(hotel_markers)



# Display figure
fig

Figure(layout=FigureLayout(height='420px'))